# Saving and loading

Get the necessary files for the next session from https://www.kaggle.com/datasets/zalando-research/fashionmnist


## Preparing the data

In [ ]:
#!pip install scikit-learn
#!pip install tensorflow==2.10
import pandas as pd
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
print("Tesorflow version {tf.__version__}")

#read data with pandas
dataTrain = pd.read_csv('data/fashion-mnist_train.csv')
dataTest = pd.read_csv('data/fashion-mnist_test.csv')
#convert to numpy and initialize variables for training
X_train, y_train = dataTrain.iloc[:, 1:].to_numpy(), dataTrain.iloc[:, 0].to_numpy()
X_test, y_test = dataTest.iloc[:, 1:].to_numpy(), dataTest.iloc[:, 0].to_numpy()

#Define Classnames
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
print(tf.config.list_physical_devices('GPU'))

# prepare ground truth as one-hot encoded values
y_one_hot = tf.one_hot(y_train,len(class_names))
y_one_hot_test = tf.one_hot(y_test,len(class_names))

# Now make the model overfitting


In [ ]:
from tensorflow.keras.optimizers.experimental import SGD
model = Sequential([
      Dense(512, activation='relu'),
      Dense(512, activation='relu'),
      Dense(10, activation="softmax")
      ])
model.compile(optimizer=SGD(learning_rate=0.001), loss="categorical_crossentropy", metrics=['accuracy'])
model.fit(
    X_test,
    y_one_hot_test,
    epochs=10,
    batch_size=100)
model.summary()


In [ ]:
print("Evaluation on Training Data")
print(model.evaluate(X_train[0:10000], y_one_hot[0:10000], batch_size=1000))

# Saving & Loading just the weights

In [ ]:
# The following craete a savedirectory if not yet available
!mkdir testmodel
!dir

In [ ]:
model.save_weights("testmodel/model1.h5",  save_format="h5")

In [ ]:
!dir testmodel

In [ ]:
# recreate the network
model2 = Sequential([
      Dense(512, activation='relu'),
      Dense(512, activation='relu'),
      Dense(10, activation="softmax")
      ])
model2.load_weights("testmodel/model1")
model2.compile(optimizer=SGD(learning_rate=0.001), loss="categorical_crossentropy", metrics=['accuracy'])




In [ ]:
print("Evaluation on Training Data")
print(model2.evaluate(X_train[0:10000], y_one_hot[0:10000], batch_size=1000))

# Saving & Loading the complete model

In [ ]:
model.save("testmodel/model2")

In [ ]:
from tensorflow import keras
model3 = keras.models.load_model("testmodel/model2")
model3.summary()

In [ ]:
print("Evaluation on Training Data")
print(model3.evaluate(X_train[0:10000], y_one_hot[0:10000], batch_size=1000))

# Saving continuously during training

ModelCheckpoints help to regularily save the model during training. This is good on cloud environments like google colab, when your model could be unexpectedly stopped to not loose already gained progress.

https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint


In [ ]:
model = Sequential([
      Dense(512, activation='relu'),
      Dense(512, activation='relu'),
      Dense(10, activation="softmax")
      ])
model.compile(
    optimizer=SGD(learning_rate=0.001), 
    loss="sparse_categorical_crossentropy", 
    metrics=['accuracy'])

from tensorflow.keras.callbacks import ModelCheckpoint;
cp_callback = ModelCheckpoint(filepath="testmodel/training",
                              save_weights_only=True,
                              verbose=1)

model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=100,
    callbacks=[cp_callback])

In [ ]:
model3.load_weights("testmodel/training")
